In [ ]:
from pathlib import Path
from genetic_algorithm import genetic_algorithm
from genetic_algorithm.selection import TournamentSelection
from mwcnf_parser import parse_mwcnf
from solver import MwcnfGenerator
from solver.fitness import satisfied_clause_count, weights
from solver.mwcnf_individual import MwcnfIndividual
from weighted_formula.weighted_formula import WeightedCnf

from io import StringIO
import matplotlib.pyplot as plt


In [ ]:
def evaluate_hyperparameters(formula_file: str | Path,
                             population_size: int,
                             generation_cnt: int,
                             tournament_size: float,
                             crossover_probability: float,
                             mutation_rate: float,
                             elitism: int,
                             print_plot: bool = True):


    debug_stream = StringIO()

    formula_path = Path(formula_file)
    formula = parse_mwcnf(formula_path)
    solution, i = genetic_algorithm(MwcnfGenerator(formula),
                                    population_size=population_size,
                                    number_of_generations=generation_cnt,
                                    selection=TournamentSelection(tournament_size),
                                    crossover_probability=crossover_probability,
                                    mutation_rate=mutation_rate,
                                    elitism=elitism,
                                    debug_stream=debug_stream if print_plot else None)

    if print_plot:
        orders = []
        bests = []
        means = []
        medians = []
        worsts = []
        for line in debug_stream.getvalue().splitlines():
            order, best, mean_, median_, worst = line.split(sep=',')
            orders.append(abs(int(order)))
            bests.append(abs(float(best)))
            means.append(abs(float(mean_)))
            medians.append(abs(float(median_)))
            worsts.append(abs(float(worst)))
        
        plt.plot(orders, bests, orders, means, orders, medians, orders, worsts)
        plt.show()

    if isinstance(solution, MwcnfIndividual):
        print(f"Number satisfied: {-satisfied_clause_count(formula, solution.config)}")
        print(f"{formula_path.name} {-weights(formula, solution.config)} {solution.config.get_evaluation()} 0")

evaluate_hyperparameters("wuf20-91/wuf20-91-Q/wuf20-01.mwcnf",
                         population_size=100,
                         generation_cnt=300,
                         tournament_size=1.75,
                         crossover_probability=0.9,
                         mutation_rate=0.01,
                         elitism=1,
                         print_plot=True)